## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
column_names = ['Education', 
                'Age', 
                'DistanceFromHome', 
                'JobSatisfaction', 
                'OverTime', 
                'StockOptionLevel',
                'WorkLifeBalance', 
                'YearsAtCompany', 
                'YearsSinceLastPromotion', 
                'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[column_names]

# Show the data types for X_df
X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=14)

In [6]:
# Convert your X data to numeric data types however you see fit
X_train['OverTime'] = X_train['OverTime'].apply(lambda over_time: 1 if over_time == 'Yes' else 0)
X_test['OverTime'] = X_test['OverTime'].apply(lambda over_time: 1 if over_time == 'Yes' else 0)

# Add new code cells as necessary
X_train['OverTime'].value_counts()

OverTime
0    791
1    311
Name: count, dtype: int64

In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Create a OneHotEncoder for the Department column
ohe_department = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
ohe_department.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = ohe_department.transform(y_train[['Department']])
y_test_department = ohe_department.transform(y_test[['Department']])

y_train_department

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [9]:
# Create a OneHotEncoder for the Attrition column
ohe_attrition = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
ohe_attrition.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = ohe_attrition.transform(y_train[['Attrition']])
y_test_attrition = ohe_attrition.transform(y_test[['Attrition']])

y_train_attrition

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data
num_columns = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_columns,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared2')(shared_layer1)

2024-07-30 07:01:16.584151: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-07-30 07:01:16.584180: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-07-30 07:01:16.584184: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-07-30 07:01:16.584227: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-30 07:01:16.584238: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(32, activation='relu', 
                                       name='department_hidden')(shared_layer2)

# Create the output layer
department_output_layer = layers.Dense(y_train_department.shape[1], 
                                       activation='softmax', 
                                       name='department_output')(department_hidden_layer)

In [12]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(32, activation='relu', 
                                      name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(y_train_attrition.shape[1], 
                                      activation='sigmoid', 
                                      name='attrition_output')(attrition_hidden_layer)

In [13]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam', 
              loss={'department_output': 'categorical_crossentropy', 
                    'attrition_output': 'binary_crossentropy'}, 
              metrics={'department_output': 'accuracy', 
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared1 (Dense)     │ (None, 64)        │        704 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared2 (Dense)     │ (None, 128)       │      8,320 │ shared1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Train the model
history = model.fit(X_train_scaled, 
                    {'department_output': y_train_department, 
                     'attrition_output': y_train_attrition},
                    epochs=100,
                    batch_size=32,
                    verbose=1)

Epoch 1/100


2024-07-30 07:01:17.261963: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - attrition_output_accuracy: 0.8064 - department_output_accuracy: 0.4388 - loss: 1.5622
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - attrition_output_accuracy: 0.8318 - department_output_accuracy: 0.6631 - loss: 1.1761
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - attrition_output_accuracy: 0.8331 - department_output_accuracy: 0.6564 - loss: 1.1539
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - attrition_output_accuracy: 0.8452 - department_output_accuracy: 0.6531 - loss: 1.1210
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - attrition_output_accuracy: 0.8518 - department_output_accuracy: 0.6378 - loss: 1.1345
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - attrition_output_accuracy: 0.8502 - department_output_accuracy: 0.6454 - loss: 1.0857
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - attrition_output_accuracy: 0.8681 - department_output_accuracy: 0.6452 - loss: 1.0775
Epoch 8/100
35/35 ━━━━━━━━━━━━━━

In [15]:
results = model.evaluate(X_test_scaled, 
                         {'department_output': y_test_department, 
                          'attrition_output': y_test_attrition}, 
                         verbose=1)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - attrition_output_accuracy: 0.7910 - department_output_accuracy: 0.5593 - loss: 4.5463


In [16]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, 
                              {'department_output': y_test_department,  
                               'attrition_output': y_test_attrition}, 
                              verbose=1)
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - attrition_output_accuracy: 0.7910 - department_output_accuracy: 0.5593 - loss: 4.5463


[4.106147289276123, 0.7989130616188049, 0.5760869383811951]

In [17]:
# Print the accuracy for both department and attrition
print(f"Department predicions accuracy: {test_results[2]}")
print(f"Attrition predictions accuracy: {test_results[1]}")

Department predicions accuracy: 0.5760869383811951
Attrition predictions accuracy: 0.7989130616188049


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is not always the best way to measure how good the model is. If most employees stay and only a few leave, the model might just predict that everyone stays to get a high accuracy. Instead, we should look at how well the model predicts the employees who actually leave. Metrics like precision (how many predicted leavers actually leave) and recall (how many actual leavers were correctly predicted) are better for this.

2. For predicting if an employee will leave (yes or no), I used the sigmoid function. This function gives a result between 0 and 1, which is perfect for yes/no questions. For predicting the best department for an employee, I used the softmax function. This function gives a probability for each department, showing which one is most likely the best fit.


3. Few ways to improve the model: 
* Add More Useful Data: Create new features from the existing data that might be more helpful for the predictions.
* Fine-Tune Settings: Adjust the model's settings like the number of layers and neurons to find the best combination.
* Prevent Overfitting: Use techniques like dropout (randomly turning off some neurons during training) or L2 regularization to make sure the model doesn't just memorize the training data.
* Test More Thoroughly: Instead of just splitting the data once, use different parts of the data to train and test the model multiple times to get a better sense of its performance.
* Combine Models: Use multiple models and combine their predictions to get better results than a single model.
* Balance the Data: If some departments have a lot more employees than others, balance the data by either adding more samples for smaller departments or creating synthetic data to even things out.